In [1]:
MAXLEN = 5

def get_all_notes(filename):
    
    f = open(filename)
    notes = set()
    for line in f.readlines():
        line = line.rstrip()
        #line = line.replace('|', ' ')
        #line = line.replace('-', '')
        #line = line.replace('#', '')
        tokens = line.split(' ')
        #tokens = set(line)
        #print(tokens)
        for token in tokens:
            notes.add(token)
        notes.add(' ')
    return notes

def get_all_lines(filename):
    f = open(filename)
    notes = []
    for line in f.readlines():
        line = line.rstrip()
        #line = line.replace('|', ' ')
        #line = line.replace('-', '')
        #line = line.replace('#', '')
        tokens = line.split(' ')
        
        if len(tokens) > MAXLEN:
            tokens = tokens[:MAXLEN]
        elif len(tokens) < MAXLEN:
            tokens = tokens + [' '] * (MAXLEN-len(tokens))
        
        notes.append(tokens)
    return notes

def clean_lines(lines):
    notes = []
    for line in lines:
        line = line.rstrip()
        line = line.replace('|', ' ')
        line = line.replace('-', '')
        line = line.replace('#', '')
        tokens = line.split(' ')
        
        if len(tokens) > MAXLEN:
            tokens = tokens[:MAXLEN]
        elif len(tokens) < MAXLEN:
            tokens = tokens + [' '] * (MAXLEN-len(tokens))
        
        notes.append(tokens)
    return notes

def get_chars(filename):
    
    f = open(filename)
    chars = set()
    for line in f:
        line = line.rstrip()
        for c in line:
            chars.add(c)
    return chars

def get_lines(filename, length):
    
    f = open(filename)
    lines = []
    for line in f:
        line = line.rstrip()
        line = line.replace('|', ' | ')
        line = line.replace('-', '')
        l = len(line)
        if l > length:
            line = line[:length]
        elif l < length:
            line = line + ' '*(length-l)
        lines.append(line)
    return lines



In [2]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            #print(c, C)
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ' '.join(self.indices_char[x] for x in x)

    


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 1
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
# MAXLEN = DIGITS + 1 + DIGITS
# MAXLEN = 10
# All the numbers, plus sign and space for padding.
chars = list('0123456789+ ')
chars = get_all_notes('answers.txt').union(get_all_notes('questions.txt'))
#hars = get_chars('answers.txt')
DIGITS = len(chars)
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
# while len(questions) < TRAINING_SIZE:
#     f = lambda: int(''.join(np.random.choice(list('0123456789'))
#                     for i in range(np.random.randint(1, DIGITS + 1))))
#     a, b = f(), f()
#     # Skip any addition questions we've already seen
#     # Also skip any such that x+Y == Y+x (hence the sorting).
#     key = tuple(sorted((a, b)))
#     if key in seen:
#         continue
#     seen.add(key)
#     # Pad the data with spaces such that it is always MAXLEN.
#     q = '{}+{}'.format(a, b)
#     query = q + ' ' * (MAXLEN - len(q))
#     ans = str(a + b)
#     # Answers can be of maximum size DIGITS + 1.
#     ans += ' ' * (DIGITS + 1 - len(ans))
#     if REVERSE:
#         # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
#         # space used for padding.)
#         query = query[::-1]
#     questions.append(query)
#     expected.append(ans)

#questions = get_lines('questions.txt', MAXLEN)
#expected = get_lines('answers.txt', MAXLEN)
questions = get_all_lines('questions.txt')
expected = get_all_lines('answers.txt')

print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, MAXLEN)
    
    
    


Using TensorFlow backend.


Generating data...
Total addition questions: 996
Vectorization...


In [3]:
chars

{'D5|D6|G5',
 'C3|C4',
 'B4|D4|D5',
 'A4|E4|G4',
 'C3|G2',
 'C5|F5',
 'C#4|C#5|E4',
 'F5',
 'D3|F4',
 'C#5|F#4|F#5',
 'C4|G4',
 'C4|F4',
 'C#4|E-4',
 'A3|A4|C4|E4',
 'A2|C#2',
 'F2|F3',
 'C3|F3|G3',
 'B4|D4|G4',
 'A-4|C5|D-5|F4|F5',
 'A5|C5|F5',
 'B3',
 'D5|G4|G5',
 'A5|C#5|E5',
 'E-3',
 'B-4|D-4|F4',
 'A-2|A-3',
 'G#2',
 'C#4|F#4',
 'C4|G3|G4',
 'C5|F#4',
 'A3|C4|E4',
 'C2|C5|E-5|G4',
 'B3|F3',
 'A-3|A-4|D4',
 'B3|D4|G2|G4',
 'B-3|C#4|E4|G3',
 'A1|D1|D2',
 'C#5|C#6|F#5',
 'B-2|E-2|E-3',
 'A1',
 'G#1|G#2',
 'B-3|D3|G3',
 'A#4',
 'B-3|D5',
 'A4|F4|F5',
 'C2|C3|C4',
 'C#4|E4',
 'D#6',
 'B-3|G3',
 'B3|E3',
 'G1',
 'A4|D5|G5',
 'A3|A4|C4|F4',
 'D-2|D-3',
 'C#5|G4|G5',
 'A3|D3',
 'C#4|F3|G#3',
 'C3|E2|E3',
 'A4|A5|F5',
 'C#5|C#6|E5',
 'A4|D5|F4|F5',
 'B-3|G3|G4',
 'A3|B-2',
 'A2|A4|C4|E3|E4',
 'D3|G3',
 'C3|E-3',
 'A3|G#4',
 'A1|G2',
 'B3|D3|D4|G2',
 'B-4|E-4|F2|G4',
 'B-3|C3',
 'G#4',
 'B3|E2|E3|E4|G3',
 'B-3|D4|G3|G4',
 'A4|E4',
 'B3|D4|G4',
 'B5|D5|D6|G5',
 'A2|G3|G4',
 'A4|D4|D5|F4',
 '

In [6]:


# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 3

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(MAXLEN))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               609792    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
lstm_6 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
lstm_7 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 1062)           136998    
_________________________________________________________________
activation_2 (Activation)    (None, 5, 1062)           0     

In [13]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

model.load_weights('musicmodelNOTELongChord.h5')
for iteration in range(1000, 1):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val), shuffle=True)
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    #model.save('musicmodelNOTEv2.h5')

Training Data:
(897, 5, 1062)
(897, 5, 1062)
Validation Data:
(99, 5, 1062)
(99, 5, 1062)


In [14]:
def generate(primer_list):
    
    questions = clean_lines(primer_list)

    #print('Total addition questions:', len(questions))

    
    x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
    #y = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(questions):
        x[i] = ctable.encode(sentence, MAXLEN)
    #for i, sentence in enumerate(expected):
    #    y[i] = ctable.encode(sentence, MAXLEN)
    preds = model.predict_classes(x)
    tmp = []
    for xrow in preds:
        tmp.append(ctable.decode(xrow, calc_argmax=False))
    
    
    
    return tmp
    

In [15]:
generate(['G2|E3|E3|G2|G3', 'A2 B3 C4 F2 F2'])

['D4 B3 B3 D4 D4', 'F2 D3 F2 D4 D4']

In [16]:
def generate_long(primer, length=5):
    
    primer_list = primer
    last = primer
    
    for i in range(length):
        last = generate([last])[0]
        primer_list += ' ' + last
    return primer_list    

In [17]:
generate_long('A3 B3 C3 D3 E3', 20)

'A3 B3 C3 D3 E3 G4 C3 C3 C3 E3 B-3 C5 B-3 B-3 B-3 A2 C5 E4 B3 C4 G4 C6 D2 G1 A3|C4|E4 F2|F3 B3|E4|G3 G#3 G4 E2|E3 E3 A4|E4 A4 F3 G3 A3 C5 C5 F5 D5 B-3 C4|G4 D4|F4 A3 A3 F3 B3|G4 E3 D4 G3 D3 D3 F3 D3 A3 C3 C4 C4 G4 B-2 G2 D4 D3 E4 G3 B3 D4 D4 B3 C3 A2 G3 E4 C3 C3 E3 C5 B3 C2 B4 G3 A3 C5 D4 F4 A4 B2 B-4 C4 C4|E4|G4 F4 E3|G2 B-4|D4|D5|F4 D4 D4 B3 B3 B3 B3 B3|E4|G4 A2 B-2 C5 F#3 B4'